In [3]:
import pandas as pd

In [1]:
chedui2luxian = {
    '博佳': ['Y5', 'Y6', 'Y8', 'Y9', 'Y10', 'Y12', 'Y13', 'Y14', 'Y16', 'Y17', 'Y21'],
    '讯致': ['Y7', 'Y18', 'Y19', 'Y20'],
    '美润': ['Y1', 'Y2', 'Y3', 'Y4', 'Y11', 'Y22']
}


In [45]:
def find_chedui(chedui_luxian, search_luxian):
    for chedui, luxian in chedui_luxian.items():
        if search_luxian in luxian:
            return chedui

In [4]:
checi_res = pd.read_excel('C:/intern/data/checi_run_bus_先过滤后查找方向.xlsx')

In [25]:
shike = pd.read_excel('C:/intern/data/docs_20220525/时刻表/shike.xlsx')
shike.drop_duplicates('车次id',inplace = True)

In [6]:
checi2chepai = pd.read_excel('C:/intern/data/docs_20220525/时刻表/duiying.xlsx')

In [61]:
def check_valid_chepai(checi, chepai_list):
    
    chepai_valid = []
    
    luxian = shike[shike['车次id']==checi]['路线'].values[0]
    chedui = find_chedui(chedui2luxian, luxian)
    luxian_all = chedui2luxian[chedui]
    checi_all = shike[shike['路线'].isin(luxian_all)]['车次id'].values
    chepai_all = checi2chepai[checi2chepai['车次id'].isin(checi_all)]['车牌号'].values
    
    for each in chepai_list:
        if each in chepai_all:
            chepai_valid.append(each)
    
    return chepai_valid
        

In [68]:
def filter_valid_chepai(chepai_predict_df):
    filtered = pd.DataFrame(columns=['车次id','预测运营'])
    for index, row in chepai_predict_df.iterrows():
        checi = row['车次id']
        try:
            chepai_predict = eval(row['预测运营'])
            filterd_chepai_predict = check_valid_chepai(checi,chepai_predict)
            filtered.loc[len(filtered.index)] = [checi, filterd_chepai_predict]
        except:
            filtered.loc[len(filtered.index)] = [checi, []]
        
    return filtered
        

In [71]:
filtered_res = filter_valid_chepai(checi_res)

In [74]:
filtered_res.to_excel('C:/intern/data/checi_run_bus_先过滤后查找方向_车队检测.xlsx',index=False)